# NumPy Vectorization

Vectorization, in simple words, means optimizing the algorithm so that it can run multiple operations from a single instruction. <br>


NumPy is all about vectorization. If you are familiar with Python, this is the main difficulty you’ll face because you’ll need to change your way of thinking and your new friends (among others) are named “vectors”, “arrays”, “views” or "ufuncs".


### my understanding

Let's say we have a list from 1 to 1 lakh, [1,2,3,4,....,99999,100000] <br>
Now, we want to divide all the numbers by 2. <br>
We can just start a for loop, but it is not optimized. <br> 
So, we can break the list by few parts and do the divison problem. And sum up the results back together, that is vectorization. <br>

**Mostly, we want to take advantage of computer's ability to process things simultaneously / parallel.**

#### Why vectorizaion works? 
1. parallel
2. only same data types reside in numpy array, unlike python list, which can store anything within
3. locality in memory same area unlike python list, which may store here and there

#### Vectorization is computing the same thing on different part of the list at the same to speed things up.

In [13]:
import numpy as np
from time import time 

In [14]:
start_time = time()

a = np.arange(250000).reshape(500,500)

end_time = time()
t= end_time - start_time
print("It took seconds",t)

It took seconds 0.0007829666137695312


In [15]:
a = np.arange(250000).reshape(500,500)
print(a)

[[     0      1      2 ...    497    498    499]
 [   500    501    502 ...    997    998    999]
 [  1000   1001   1002 ...   1497   1498   1499]
 ...
 [248500 248501 248502 ... 248997 248998 248999]
 [249000 249001 249002 ... 249497 249498 249499]
 [249500 249501 249502 ... 249997 249998 249999]]


# Object Oriented Approach 

Let’s take a very simple example, a random walk. One possible object-oriented approach would be to define a RandomWalker class and write a walk method that would return the current position after each (random) step. It’s nice, it’s readable, but it is slow:

In [17]:
from tools import timeit #get time it from tools.py(custom module)
import random
class RandomWalker:
  def __init__(self):
           self.position = 0
  def walk(self, n): # walk method
    self.position = 0
    for i in range(n):
      yield self.position 
      self.position += 2*random.randint(0, 1) - 1
      #returns current position after each random step
           
walker = RandomWalker() # make instance of class walk
walk = [position for position in walker.walk(1000)]#call the walk function

walker = RandomWalker()
timeit("[position for position in walker.walk(n=10000)]", globals())
#calculates the  total loops and time per loop

10 loops, best of 3: 4.86 msec per loop


# Procedural Approach

For such a simple problem, we can probably save the class definition and concentrate only on the walk method that computes successive positions after each random step. This new method saves some CPU cycles but not that much because of this function is pretty much the same as in the object-oriented approach and the few cycles we saved probably come from the inner Python object-oriented machinery.

In [18]:
from tools import timeit #get timeit from tools.py (custom module)
import random
def random_walk(n):
    position = 0
    walk = [position]
    for i in range(n):
        position += 2*random.randint(0, 1)-1 #position takes up random values 
        walk.append(position)# append position to walk
    return walk

walk = random_walk(1000) #call the function random_walk
timeit("random_walk(n=10000)", globals()) # calculates the total loops and time per loop

10 loops, best of 3: 4.54 msec per loop


# Vectorized Approach

1. Itertools
   Itertools is a python module that offers a set of functions creating iterators for efficient looping. If we observe that a random walk is an accumulation of steps, we can rewrite the function by first generating all the steps and accumulate them without any loop:

In [19]:
from tools import timeit #get timeit function from tools.py(custom module)
from itertools import accumulate #get accumulate function from built accumulate module
import random 
def random_walk_faster(n=1000):
  steps = random.choices([-1,+1], k=n)
  return [0]+list(accumulate(steps))#get the total number of steps

walk = random_walk_faster(1000) 
timeit("random_walk_faster(n=10000)", globals())# calculates the total loops and time per loop

100 loops, best of 3: 848 usec per loop


In fact, we’ve just vectorized our function. Instead of looping for picking sequential steps and add them to the current position, we first generated all the steps at once and used the accumulate function to compute all the positions. We got rid of the loop and this makes things faster.

2. Numpy

   We gained 85% of computation-time compared to the previous version, not so bad. But the advantage of this new version is that it makes NumPy vectorization super simple. We just have to translate itertools call into NumPy ones:
   

In [20]:
from tools import timeit #get timeit function from tools.py(custom module)
import numpy as np
def random_walk_fastest(n=1000):
    # No 's' in NumPy choice (Python offers choice & choices)
    steps = np.random.choice([-1,+1], n)
    return np.cumsum(steps) #return the cumulative sum of the steps along a given axis.

walk = random_walk_fastest(1000)
timeit("random_walk_fastest(n=1000)", globals())
#calculates the total loops and time per loop

1000 loops, best of 3: 18.9 usec per loop
